In [1]:
# 4/8/17 Stage 2 code
import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
import mxnet as mx
import pandas as pd
from sklearn import cross_validation
import datetime as dt
# import cloudpickle as pickle

os.chdir('f:/dsb/inception_bn')
# get path in which file resides
filePath = os.path.dirname(os.path.realpath(os.getcwd()))
# replace xgboost with lightgbm
from lightgbm.sklearn import LGBMRegressor
from lightgbm.sklearn import LGBMClassifier

#Some helper methods

def getFiles(aDir):
    aList = []
    for file in os.listdir(aDir):
        aList.append(file)
    return(aList)

def FileExists(aFile, aDir):
    aList = getFiles(aDir)
    
    for fl in aList:
        if fl == aFile:
            return True
        
    return False

def get_extractor():
    model = mx.model.FeedForward.load('model/inception-bn', 126, ctx=mx.cpu(), numpy_batch_size=1)
    fea_symbol = model.symbol.get_internals()["flatten_output"]
    feature_extractor = mx.model.FeedForward(ctx=mx.cpu(), symbol=fea_symbol, numpy_batch_size=64,
                                             arg_params=model.arg_params, aux_params=model.aux_params,
                                             allow_extra_params=True)

    return feature_extractor


def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])


def get_data_id(path):
    sample_image = get_3d_data(path)
    sample_image[sample_image == -2000] = 0
    # f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))

        # if cnt < 20:
        #     plots[cnt // 5, cnt % 5].axis('off')
        #     plots[cnt // 5, cnt % 5].imshow(np.swapaxes(tmp, 0, 2))
        # cnt += 1

    # plt.show()
    batch = np.array(batch)
    return batch


def calc_features():
    net = get_extractor()
    for folder in glob.glob('stage1/*'):
        batch = get_data_id(folder)
        feats = net.predict(batch)
        print(feats.shape)
        np.save(folder, feats)


def train_xgboost():
    df = pd.read_csv('data/stage2_labels.csv')
    print(df.head())

    x = np.array([np.mean(np.load('stage1/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
    y = df['cancer'].as_matrix()

    trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                   test_size=0.20)

# replaced xgb with lightgbm
    clf = LGBMClassifier(max_depth=50,
                        num_leaves=21,
                        n_estimators=5000,
                        min_child_weight=1,
                        learning_rate=0.001,
                        nthread=24,
                        subsample=0.80,
                        colsample_bytree=0.80,
                        seed=42)

    clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=True, eval_metric='logloss', early_stopping_rounds=300)

    return clf


def make_submit():
    # if not FileExists('xgModel.pk1', filePath):
        #Train model then serialize
    clf = train_xgboost()
    #    with open(filePath + '/' + 'xgModel.pk1','wb') as objSave:
    #        pickle.dump(clf, objSave, pickle.HIGHEST_PROTOCOL)
    # else:
    #    with open(filePath + '/' + 'xgModel.pk1','rb') as objLoad:
            #if model exists, just load from file
    #        clf = pickle.load(objLoad)

    df = pd.read_csv('data/stage2_sample_submission.csv')

    x = np.array([np.mean(np.load('stage1/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

    pred = clf.predict(x)

    df['cancer'] = pred
    df.to_csv('stage2_subm_inception_logloss.csv', index=False)
    print(df.head())

C:\Anaconda\envs\py35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.getcwd()

'f:\\dsb\\inception_bn'

In [4]:
if __name__ == '__main__':
    print(dt.datetime.now())
    calc_features()
    # ~20h
    print(dt.datetime.now())

2017-04-08 14:56:43.658099
(159, 1024)
(136, 1024)
(160, 1024)
(44, 1024)
(154, 1024)
(174, 1024)
(132, 1024)
(179, 1024)
(153, 1024)
(119, 1024)
(119, 1024)
(149, 1024)
(157, 1024)
(94, 1024)
(111, 1024)
(162, 1024)
(35, 1024)
(106, 1024)
(132, 1024)
(129, 1024)
(113, 1024)
(87, 1024)
(162, 1024)
(110, 1024)
(93, 1024)
(36, 1024)
(173, 1024)
(53, 1024)
(167, 1024)
(140, 1024)
(140, 1024)
(43, 1024)
(145, 1024)
(40, 1024)
(153, 1024)
(108, 1024)
(180, 1024)
(84, 1024)
(110, 1024)
(43, 1024)
(36, 1024)
(169, 1024)
(118, 1024)
(120, 1024)
(158, 1024)
(152, 1024)
(112, 1024)
(112, 1024)
(35, 1024)
(112, 1024)
(126, 1024)
(154, 1024)
(114, 1024)
(140, 1024)
(40, 1024)
(83, 1024)
(103, 1024)
(167, 1024)
(148, 1024)
(114, 1024)
(109, 1024)
(147, 1024)
(105, 1024)
(112, 1024)
(142, 1024)
(190, 1024)
(110, 1024)
(186, 1024)
(141, 1024)
(163, 1024)
(122, 1024)
(144, 1024)
(110, 1024)
(118, 1024)
(103, 1024)
(118, 1024)
(149, 1024)
(123, 1024)
(112, 1024)
(177, 1024)
(91, 1024)
(38, 1024)
(144, 

In [3]:
print(dt.datetime.now())
make_submit()
print(dt.datetime.now())

2017-04-09 22:56:00.021454
                                 id  cancer
0  0015ceb851d7251b8f399e39779d1e7d       1
1  0030a160d58723ff36d73f41b170ec21       0
2  003f41c78e6acfa92430a057ac0b306e       0
3  006b96310a37b36cccb2ab48d10b49a3       1
4  008464bb8521d09a42985dd8add3d0d2       1
[1]	valid_0's binary_logloss: 0.692884
Train until valid scores didn't improve in 300 rounds.
[2]	valid_0's binary_logloss: 0.692707
[3]	valid_0's binary_logloss: 0.692437
[4]	valid_0's binary_logloss: 0.692177
[5]	valid_0's binary_logloss: 0.691933
[6]	valid_0's binary_logloss: 0.691738
[7]	valid_0's binary_logloss: 0.691485
[8]	valid_0's binary_logloss: 0.691228
[9]	valid_0's binary_logloss: 0.690994
[10]	valid_0's binary_logloss: 0.690748
[11]	valid_0's binary_logloss: 0.690497
[12]	valid_0's binary_logloss: 0.690204
[13]	valid_0's binary_logloss: 0.689946
[14]	valid_0's binary_logloss: 0.689705
[15]	valid_0's binary_logloss: 0.68948
[16]	valid_0's binary_logloss: 0.689254
[17]	valid_0's binary_lo